# Pure clustering

Test clustering algorithm that yields clusters exceeding a given "purity" level.

In [9]:
import sys
sys.path.insert(0,"..")
import time
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import speasy as spz
import sklearn
import os
import pickle as pkl
from sklearn.metrics import *
from astropy import constants as Cst
from sklearn import preprocessing



# locally defined functions
import speasy_utils
from speasy_utils.coord import *
from speasy_utils.plot import *

# default format for datetime objects
date_fmt="%Y-%m-%dT%H:%M:%S"
# excess time for interpolation
delta_t = timedelta(days=1)
# interpolation freq
freq = "60S"
# ACE data timeshift
shift_t = timedelta(seconds=4000)

We define some labels for each of classes we wish to detect as well as the associated color mapping.

In [10]:
# magnetoshere regions
region_labels = ["SW", "MgtSth", "MgtSph"]
# color mapping used
cmap = mpl.colors.ListedColormap(['r','g','b'])

Time range of the data used for training and testing the model. The data provided by AMDA has variable sampling time. We interpolate each feature with a fixed sampling time. To avoid side effect we retrieve data from `start - delta_t` to `stop + delta_t`. 

In [11]:
#start, stop = "2007-03-01T00:00:00", "2008-03-01T00:00:00"
start, stop = "2007-03-01T00:00:00", "2009-04-01T00:00:00"
start, stop = datetime.strptime(start, date_fmt), \
    datetime.strptime(stop, date_fmt)
# used for naming data file
start_str, stop_str = start.strftime(date_fmt),stop.strftime(date_fmt)
# start and stop for getting data
start_d, stop_d = start - delta_t, stop + delta_t

Ids of the parameters measured by the satellite:
- `sat_b_id` : id of the satellite's magnetic field measurements (GSE) (nT)
- `sat_n_id` : id of the satellite's ion density measurements 
- `sat_x_id` : id of the satellite's position data (GSE) (Re)

In [12]:
# Themis B - Prediction parameters
sat_b_id = "thb_bs" # magnetic field mesurements in GSE coordinates
sat_n_id = "thb_n_i" # plasma ion density
sat_x_id = "thb_xyz" # target position in GSE coordinates

# Themis A - prediction parameters
#target_b = "tha_bs"
#target_n = "tha_n_i"
#target_x = "tha_xyz"

Ids of the solar wind parameters : 
- `sw_b_id` : id of solar wind magnetic field measurements (GSE) (nT)
- `sw_n_id` : id of solar wind ion density measurements 
- `sw_v_id` : id of solar wind speed (GSE) (km/s)
- `sw_pdyn_id` : id of solar wind ram pressure (nP)

In [13]:
# Solar wind measurements (AMDA/OMNI data)
sw_b_id = "omni1_imf"
sw_n_id = "omni1_sw_n"
sw_v_id = "omni1_sw_vgse"
sw_pdyn_id = "omni1_sw_pdyn"

Get the interpolated data.

In [17]:
data_filename = f"parameter_data_{start_str}_{stop_str}.pkl"

parameter_ids = [sw_b_id,sw_n_id,sw_v_id,sw_pdyn_id] + \
                [sat_b_id, sat_n_id, sat_x_id]

m = len(parameter_ids)
parameter_shifts = {}#{"imf":shift_t, "sw_n":shift_t}
from speasy_utils.amda import *
t, interpolated_data, bounds = get_interpolated_data(parameter_ids, 
                                    start_d,
                                    stop_d,
                                    data_filename, 
                                    shifts=parameter_shifts,
                                    freq=freq)

print(f"Interpolated data: ")
print(f"\tstart : {bounds[0]}")
print(f"\tstop  : {bounds[1]}")
print(f"\tfreq  : {freq}")
print(f"\tdata shapes")
for k,v in interpolated_data.items():
    print(f"\t\t{k}: {v.shape}")

KeyboardInterrupt: 

Set asside the solar wind parameters for later use.

In [ ]:
# Solar wind data
sw_n = interpolated_data[sw_n_id]
sw_b = interpolated_data[sw_b_id]
sw_v = interpolated_data[sw_v_id]
sw_pdyn = interpolated_data[sw_pdyn_id]

And the satellite measurements.

In [ ]:
# Satellite measurements
sat_n = interpolated_data[sat_n_id]
sat_b = interpolated_data[sat_b_id]
sat_xyz = interpolated_data[sat_x_id]

## Feature analysis

Let's check out the data. The two features we are interested in are:
- `r_b` = $|b_{sat}|  /  |b_{sat}|$
- `r_n` = $n_{sat}  /  n_{sw}$

In [ ]:
sw_B = np.linalg.norm(sw_b, axis=1) # solar wind magnetic field magnitude
sat_B = np.linalg.norm(sat_b, axis=1) # satellite magnetic field magnitude

r_b = sat_B / sw_B
r_n = (sat_n / sw_n).flatten()

Create the feature array and compute the position bounds (useful for later plots).

In [ ]:
# features
x_full = np.vstack((r_n,r_b)).T


# plot bounds for position in (X,Y) plane
xmin,xmax = sat_xyz[:,0].min(), sat_xyz[:,0].max()
ymin,ymax = sat_xyz[:,1].min(), sat_xyz[:,1].max()

Notice that `r_b` and `r_n` are concentrated around 0. Let's try transforming the data to make it easier to identify the classes.

In [ ]:
x_full = np.log(1+x_full)

fig = plot_features(x_full, xlabel="log(1+r_n)", ylabel="log(1+r_b)")
plt.show()

### Retrieving the labels
We compute the labels. Use the Shue1997 and Jerab2005 models for calculating the position of magnetopause and bow shock. Generating all the labels can take some time and we store the results in a local file to be reused on later executions.

In [ ]:
# labels
labels_data_filename = data_filename.split(".")[0]+"_y.pkl"
if not os.path.exists(labels_data_filename):
    y_full = get_regions_dyn(sat_xyz,
                           sw_pdyn,
                           sw_b[:,2],
                           sw_n,
                           np.linalg.norm(sw_v, axis=1),
                           np.linalg.norm(sw_b, axis=1))
    pkl.dump(y_full, open(labels_data_filename,"wb"))
else:
    print(f"Loading labels from {labels_data_filename}")
    y_full = pkl.load(open(labels_data_filename, "rb"))


### Scalling the features

In [ ]:
# scale the data
scaler = preprocessing.RobustScaler()
scaler.fit(x_full)
x_full = scaler.transform(x_full)

## Clustering


In [ ]:

def cluster_purity(y):
    u,c = np.unique(y, return_counts=True)
    return np.max(c)/np.sum(c), u[np.argmax(c)].astype(int)

def my_clustering(x, y, cluster_f, n_clusters=3, tol=.9, max_n_clusters=10):
    print(f"my_clustering : {x.shape}, {y.shape}, {n_clusters}")
    if x.shape[0] <= n_clusters:
        return None, None
    if n_clusters > max_n_clusters:
        return None, None
    features, labels = np.array([]),np.array([])
    rest_feat, rest_labels = np.array([]),np.array([])
    # start clustering
    clust = cluster_f(n_clusters=n_clusters)
    y_pred = clust.fit_predict(x)
    classes = set(y_pred)
    print("\tclasses : ", len(classes))
    if len(classes) < n_clusters:
        return None, None
    recluster = []
        
    for cl in classes:
        purity, cls = cluster_purity(y[y_pred==cl])
        print("\t", purity, cls, np.sum(y_pred==cl))
        if purity>=tol:
            new_labels = np.ones(np.sum(y_pred==cl))*cls
            new_feats = x[y_pred==cl]
            features = _stack_arrays(features, new_feats)
            labels = _stack_arrays(labels, new_labels)
        else:
            # set asside the rest of the data
            new_labels = y[y_pred==cl]
            new_feats = x[y_pred==cl]
            recluster.append((new_feats, new_labels))
            #f, l = my_clustering(new_feats, new_labels, cluster_f, n_clusters=n_clusters+1, tol=tol, max_n_clusters=max_n_clusters)
            #rest_feat = _stack_arrays(rest_feat, new_feats)
            #rest_labels = _stack_arrays(rest_labels, new_labels)
            #features = _stack_arrays(features, f)
            #labels = _stack_arrays(labels, l)
    for nf,nl in recluster:
        f, l = my_clustering(nf, nl, cluster_f, n_clusters=n_clusters+1, tol=tol, max_n_clusters=max_n_clusters)
        features = _stack_arrays(features, f)
        labels = _stack_arrays(labels, l)
    #if rest_feat.size and rest_feat.shape[0]>n_clusters+1:
    #    new_feats, new_labels = my_clustering(rest_feat, rest_labels, cluster_f, n_clusters=n_clusters+1, tol=tol, max_n_clusters=max_n_clusters)
    #    features = _stack_arrays(features, new_feats)
    #    labels = _stack_arrays(labels, new_labels)
    return features, labels

def my_clustering2(x, y, cluster_f, tol=.98, min_size=None, K=2):
    # number of classes in the data
    n_classes = len(set(y))
    #print(set(y))
    if min_size is None:
        min_size=int(.1 * (x.shape[0]/(K*n_classes)))
    if x.shape[0] < min_size:
        return None, None
    #print(f"my_clustering2 : shape: {x.shape}, n_classes: {n_classes}, min_size: {min_size}")
    if x.shape[0] <= K * n_classes:
        return None, None
    if n_classes == 1: 
        return x, y

    features, labels = np.array([]),np.array([])
    rest_feat, rest_labels = np.array([]),np.array([])
    
    
    
    # start clustering
    clust = cluster_f(n_clusters=K * n_classes)
    y_pred = clust.fit_predict(x)
    classes = set(y_pred)
    
    #print("\tclasses : ", len(classes))
    if len(classes) < n_classes:
        return None, None
    recluster = []
        
    for cl in classes:
        purity, cls = cluster_purity(y[y_pred==cl])
        #print("\t", purity, cls, np.sum(y_pred==cl))
        if purity>=tol:
            new_labels = (np.ones(np.sum(y_pred==cl))*cls).astype(int)
            new_feats = x[y_pred==cl]
            features = _stack_arrays(features, new_feats)
            labels = _stack_arrays(labels, new_labels).astype(int)
        else:
            # set asside the rest of the data
            new_labels = y[y_pred==cl].astype(int)
            new_feats = x[y_pred==cl]
            
            if new_labels.shape[0] >= min_size:
                recluster.append((new_feats, new_labels))
            
    for nf, nl in recluster:
        f, l = my_clustering2(nf, nl, cluster_f, tol=tol, min_size=min_size)
        features = _stack_arrays(features, f)
        labels = _stack_arrays(labels, l).astype(int)

    return features, labels  

from sklearn.cluster import KMeans, Birch
from sklearn.cluster import SpectralClustering
from cluster_utils import _stack_arrays

m = int(x_full.shape[0]/100)
rindx = np.random.choice(range(x_full.shape[0]), size=5000)

f, l= my_clustering2(x_full[rindx], y_full[rindx], SpectralClustering)

plt.hist(y_full[rindx], bins=100, density=True)

fig, axes = plt.subplots(2,1)
axes[0].scatter(f[:,0], f[:,1], c=l, alpha=.3, marker=".")
axes[1].hist(l, bins=100, density=True)


f, l= my_clustering2(x_full[rindx], y_full[rindx], SpectralClustering, K=3, tol=.9)

fig, axes = plt.subplots(2,1)
axes[0].scatter(f[:,0], f[:,1], c=l, alpha=.3, marker=".")
axes[1].hist(l, bins=100, density=True)
plt.show()

In [ ]:
n_classes = []
tols = np.linspace(0.9, 0.99, 10)
for p in tols:
    print(p)
    f, l= my_clustering2(x_full[rindx], y_full[rindx], SpectralClustering, K=2, tol=p)
    n_classes.append(len(set(l)))
    

plt.plot(tols, n_classes)
plt.show()